In [ ]:
import heapq

# A* Search Algorithm implementation
def a_star(graph, start, goal, heuristic):
    # Priority queue for nodes to explore (min-heap based on f = g + h)
    # Stores tuples of (f_cost, node_name)
    open_list = []
    heapq.heappush(open_list, (0 + heuristic[start], start)) # Push (f, start)

    # g_cost: cost from start to a node
    # We only know the start node's cost (0)
    g_cost = {node: float('inf') for node in graph}
    g_cost[start] = 0

    # parent: tracks the path
    parent = {start: None}

    while open_list:
        # Pop node with lowest f value (f = g + h)
        current_f, current_node = heapq.heappop(open_list)

        # If this path to current_node is already worse than one we've found,
        # skip it. This is a crucial optimization.
        if current_f > g_cost[current_node] + heuristic[current_node]:
             continue

        if current_node == goal:
            # Reconstruct path
            path = []
            while current_node:
                path.append(current_node)
                current_node = parent[current_node]
            return path[::-1]  # Reverse path

        for neighbor, cost in graph[current_node].items():
            # Calculate tentative g_cost (cost from start to neighbor through current)
            tentative_g = g_cost[current_node] + cost

            # If this path to neighbor is better than any previous one
            if tentative_g < g_cost[neighbor]:
                g_cost[neighbor] = tentative_g  # Update g_cost
                f = tentative_g + heuristic[neighbor] # Calculate f_cost
                heapq.heappush(open_list, (f, neighbor)) # Push to priority queue
                parent[neighbor] = current_node # Update parent

    return None  # No path found

# --- Example Data (Unchanged) ---

# Example real-life graph: Cities connected by distances (in km)
graph = {
    'A': {'B': 6, 'D': 1},
    'B': {'A': 6, 'C': 5, 'D': 2, 'E': 2},
    'C': {'B': 5, 'E': 5},
    'D': {'A': 1, 'B': 2, 'E': 1},
    'E': {'B': 2, 'C': 5, 'D': 1}
}

# Heuristic (straight-line distance to goal 'C')
heuristic = {
    'A': 7,
    'B': 6,
    'C': 0,  # Goal
    'D': 3,
    'E': 1
}

# --- User Input Section ---

# Get a set of all valid node names
valid_nodes = set(graph.keys())
print(f"Available nodes: {', '.join(sorted(valid_nodes))}")

# Get and validate Start Node
while True:
    start_node = input("Enter the start node: ").upper()
    if start_node in valid_nodes:
        break
    print(f"Invalid node. Please choose from: {', '.join(sorted(valid_nodes))}")

# Get and validate Goal Node
while True:
    goal_node = input("Enter the goal node: ").upper()
    if goal_node in valid_nodes:
        break
    print(f"Invalid node. Please choose from: {', '.join(sorted(valid_nodes))}")

# Run A* search
print(f"\nSearching for path from {start_node} to {goal_node}...")
path = a_star(graph, start_node, goal_node, heuristic)

# Print the result
if path:
    print("Optimal Path:", " -> ".join(path))
else:
    print(f"No path found from {start_node} to {goal_node}.")

Available nodes: A, B, C, D, E
Enter the start node: A
Enter the goal node: E

Searching for path from A to E...
Optimal Path: A -> D -> E


In [ ]:
from collections import deque

# BFS to find shortest path in a maze
def bfs_shortest_path(maze, start, goal):
    rows, cols = len(maze), len(maze[0])
    queue = deque([(start, [start])])  # store (position, path)
    visited = set([start])

    # Movement directions -> up, down, left, right
    directions = [(-1,0), (1,0), (0,-1), (0,1)]

    while queue:
        (x, y), path = queue.popleft()

        # Check if goal reached
        if (x, y) == goal:
            return path

        # Explore neighbors
        for dx, dy in directions:
            nx, ny = x + dx, y + dy

            # Check valid move
            if 0 <= nx < rows and 0 <= ny < cols and maze[nx][ny] == 0 and (nx, ny) not in visited:
                visited.add((nx, ny))
                queue.append(((nx, ny), path + [(nx, ny)]))

    return None  # No path found

# --- NEW FUNCTION ---
# This function prints the maze with the path
def print_maze_with_path(maze, path, start, goal):
    """
    Prints a visual representation of the maze with the path.
    - S: Start
    - G: Goal
    - #: Obstacle
    - .: Empty space
    - *: Path
    """
    # Create a visual copy of the maze, replacing 0s with '.' and 1s with '#'
    visual_maze = []
    for row in maze:
        visual_maze.append(['.' if cell == 0 else '#' for cell in row])

    # Overlay the path on the visual maze
    if path:
        for (x, y) in path:
            if (x, y) == start:
                visual_maze[x][y] = 'S'
            elif (x, y) == goal:
                visual_maze[x][y] = 'G'
            else:
                visual_maze[x][y] = '*'

    # Print the visual maze row by row
    for row in visual_maze:
        print(" ".join(row)) # Use " " to add spacing for clarity

# --- END NEW FUNCTION ---

# Example Maze (0 = free, 1 = obstacle)
maze = [
    [0, 1, 0, 0, 0],
    [0, 1, 0, 1, 0],
    [0, 0, 0, 1, 0],
    [0, 1, 1, 0, 0],
    [0, 0, 0, 0, 0]
]

start = (0, 0)
goal = (4, 4)

# --- MODIFIED MAIN SCRIPT ---
path = bfs_shortest_path(maze, start, goal)

if path:
    print("Shortest Path (coordinates):", path)
    print("Path Length:", len(path) - 1)
    print("\n--- Visualized Path ---")
    print_maze_with_path(maze, path, start, goal)
else:
    print("No path found.")

Shortest Path (coordinates): [(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4)]
Path Length: 8

--- Visualized Path ---
S # . . .
* # . # .
* . . # .
* # # . .
* * * * G


In [ ]:
# Depth-First Search (DFS) to find one path to a target
def dfs_find_one_path(graph, start, target, visited=None, path=None):
    """
    Performs a DFS traversal to find the first path to the target.
    Visualizes traversal by printing.
    """
    if visited is None:
        visited = set()
    if path is None:
        path = []

    # Mark as visited and add to our current path
    visited.add(start)
    path = path + [start] # Create a new list for this path branch

    print(f"Visiting: {start}") # Visualize the traversal order

    # Goal check
    if start == target:
        return path  # Found the target, return the path

    # Recurse for neighbors
    for neighbor in graph[start]:
        if neighbor not in visited:
            # Continue the search from the neighbor
            new_path = dfs_find_one_path(graph, neighbor, target, visited, path)
            if new_path:
                return new_path # If the recursive call found the path, pass it up

    return None # No path found from this branch

# Example game map (graph representation)
game_map = {
    'A': ['B', 'C'],
    'B': ['D', 'E'],
    'C': ['F'],
    'D': [],
    'E': ['F'],
    'F': []
}

start_node = 'A'
target_node = 'F'

print(f"--- Finding ONE path from {start_node} to {target_node} ---")
path = dfs_find_one_path(game_map, start_node, target_node)

if path:
    print("\nTarget Found!")
    print("Traversal Order (to target):", " -> ".join(path))
else:
    print(f"\nTarget {target_node} not reachable from {start_node}.")



# Depth-First Search (DFS) using backtracking to find ALL paths
def dfs_find_all_paths(graph, start, target, path=None):
    """
    Explores all possible paths from start to target using DFS/backtracking.
    """
    if path is None:
        path = []

    # Add the current node to this specific path
    path = path + [start]

    # Visualize the path being explored
    print(f"Exploring: {' -> '.join(path)}")

    # Base Case: We found the target
    if start == target:
        return [path]  # Return this path as a list

    # --- Recursive Step ---
    all_paths = [] # To store all paths found from this point

    for neighbor in graph[start]:
        # Avoid cycles by checking if we've already been here *on this path*
        if neighbor not in path:
            # Recurse and get all paths starting from the neighbor
            new_paths = dfs_find_all_paths(graph, neighbor, target, path)

            # Add any complete paths found to our list
            for p in new_paths:
                all_paths.append(p)

    return all_paths # Return all paths found from this node

# --- Run the "Find All Paths" version ---
print(f"\n--- Finding ALL paths from {start_node} to {target_node} ---")
all_paths = dfs_find_all_paths(game_map, start_node, target_node)

print("\n--- All Paths Found ---")
if all_paths:
    for i, p in enumerate(all_paths):
        print(f"Path {i+1}: {' -> '.join(p)}")
else:
    print(f"No paths found from {start_node} to {target_node}.")

--- Finding ONE path from A to F ---
Visiting: A
Visiting: B
Visiting: D
Visiting: E
Visiting: F

Target Found!
Traversal Order (to target): A -> B -> E -> F

--- Finding ALL paths from A to F ---
Exploring: A
Exploring: A -> B
Exploring: A -> B -> D
Exploring: A -> B -> E
Exploring: A -> B -> E -> F
Exploring: A -> C
Exploring: A -> C -> F

--- All Paths Found ---
Path 1: A -> B -> E -> F
Path 2: A -> C -> F


In [ ]:
import heapq

def a_star_maze(maze, start, goal):
    rows, cols = len(maze), len(maze[0])

    def heuristic(a, b):
        # Manhattan distance heuristic
        return abs(a[0] - b[0]) + abs(a[1] - b[1])

    open_list = []
    heapq.heappush(open_list, (0 + heuristic(start, goal), 0, start, [start]))
    visited = set()

    while open_list:
        f, g, current, path = heapq.heappop(open_list)
        if current == goal:
            return path

        if current in visited:
            continue
        visited.add(current)

        for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]:
            nx, ny = current[0] + dx, current[1] + dy
            if 0 <= nx < rows and 0 <= ny < cols and maze[nx][ny] == 0:
                new_cost = g + 1
                heapq.heappush(open_list, (new_cost + heuristic((nx, ny), goal), new_cost, (nx, ny), path + [(nx, ny)]))

    return None

# Example Maze
maze = [
    [0, 1, 0, 0],
    [0, 1, 0, 1],
    [0, 0, 0, 0],
    [1, 1, 1, 0]
]

start = (0, 0)
goal = (3, 3)

path = a_star_maze(maze, start, goal)
print("Optimal Path:", path)
print("Path Length:", len(path) - 1)

Optimal Path: [(0, 0), (1, 0), (2, 0), (2, 1), (2, 2), (2, 3), (3, 3)]
Path Length: 6


In [ ]:
import heapq

def a_star_maze(maze, start, goal):
    rows, cols = len(maze), len(maze[0])

    def heuristic(a, b):
        # Manhattan distance heuristic
        #
        return abs(a[0] - b[0]) + abs(a[1] - b[1])

    open_list = []
    # (f_cost, g_cost, current_node, path_list)
    heapq.heappush(open_list, (0 + heuristic(start, goal), 0, start, [start]))

    # visited: Stores (row, col) tuples
    visited = set()

    while open_list:
        # Get the node with the lowest f_cost
        f, g, current, path = heapq.heappop(open_list)

        # Check if we've already processed this node
        if current in visited:
            continue
        visited.add(current) # Mark as visited

        # --- Goal Check ---
        if current == goal:
            return path  # Found the optimal path

        (x, y) = current

        # Explore neighbors (up, down, left, right)
        for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]:
            nx, ny = x + dx, y + dy

            # Check bounds and obstacles
            if 0 <= nx < rows and 0 <= ny < cols and maze[nx][ny] == 0:
                # This is a valid, unvisited neighbor
                new_g = g + 1
                new_f = new_g + heuristic((nx, ny), goal)
                heapq.heappush(open_list, (new_f, new_g, (nx, ny), path + [(nx, ny)]))

    return None # No path found

# --- Visualization Function ---
def print_maze_with_path(maze, path, start, goal):
    """
    Prints a visual representation of the maze with the path.
    - S: Start
    - G: Goal
    - #: Obstacle
    - .: Empty space
    - *: Path
    """
    # Create a visual copy of the maze, replacing 0s with '.' and 1s with '#'
    visual_maze = []
    for row in maze:
        visual_maze.append(['.' if cell == 0 else '#' for cell in row])

    # Overlay the path on the visual maze
    if path:
        for (x, y) in path:
            if (x, y) == start:
                visual_maze[x][y] = 'S'
            elif (x, y) == goal:
                visual_maze[x][y] = 'G'
            else:
                visual_maze[x][y] = '*'

    # Print the visual maze row by row
    print("--- Visualized Path ---")
    for row in visual_maze:
        print(" ".join(row)) # Use " " to add spacing for clarity

# --- Main Execution ---

# Example Maze (0 = free, 1 = obstacle)
maze = [
    [0, 1, 0, 0],
    [0, 1, 0, 1],
    [0, 0, 0, 0],
    [1, 1, 1, 0]
]

start = (0, 0)
goal = (3, 3)

path = a_star_maze(maze, start, goal)

if path:
    print("Optimal Path (coordinates):", path)
    print("Path Length:", len(path) - 1)
    print_maze_with_path(maze, path, start, goal)
else:
    print(f"No path found from {start} to {goal}.")

Optimal Path (coordinates): [(0, 0), (1, 0), (2, 0), (2, 1), (2, 2), (2, 3), (3, 3)]
Path Length: 6
--- Visualized Path ---
S # . .
* # . #
* * * *
# # # G


In [ ]:
import heapq

def heuristic(state, goal):
    # Manhattan distance for each tile
    return sum(abs(s % 3 - g % 3) + abs(s // 3 - g // 3)
               for s, g in zip(state, goal) if s != 0)

def get_neighbors(state):
    neighbors = []
    idx = state.index(0) # Find the index of the empty spot (0)
    moves = []

    # Check possible moves (up, down, left, right)
    if idx not in [0, 1, 2]: moves.append(-3) # Up
    if idx not in [6, 7, 8]: moves.append(3)  # Down
    if idx not in [0, 3, 6]: moves.append(-1) # Left
    if idx not in [2, 5, 8]: moves.append(1)  # Right

    for move in moves:
        new_state = state[:]
        # Swap the empty spot with the tile
        new_state[idx], new_state[idx + move] = new_state[idx + move], new_state[idx]
        neighbors.append(new_state)
    return neighbors

def a_star_8_puzzle(start, goal):
    # open_list stores (f_cost, g_cost, state, path)
    open_list = []
    # (f, g, state, path)
    heapq.heappush(open_list, (heuristic(start, goal), 0, start, []))

    visited = set() # Stores states we've already processed

    while open_list:
        f, g, current, path = heapq.heappop(open_list)

        # --- Goal Check ---
        if current == goal:
            return path + [current] # Return the complete path

        # --- Visited Check ---
        # If we've seen this state before, skip it
        if tuple(current) in visited:
            continue
        visited.add(tuple(current)) # Mark as visited

        # --- Explore Neighbors ---
        for neighbor in get_neighbors(current):
            if tuple(neighbor) not in visited:
                new_g = g + 1
                new_h = heuristic(neighbor, goal)
                new_f = new_g + new_h
                heapq.heappush(open_list, (new_f, new_g, neighbor, path + [current]))

    return None # No solution found

# --- VISUALIZATION FUNCTION ---
def visualize_puzzle(state):
    """Prints a 3x3 grid for a given puzzle state."""
    print("-------")
    for i in range(3):
        row = state[i*3 : i*3 + 3]
        # Format the row, replacing 0 with a blank space
        print(f"| {row[0] if row[0] != 0 else ' '} {row[1] if row[1] != 0 else ' '} {row[2] if row[2] != 0 else ' '} |")
    print("-------")

# --- MAIN SCRIPT ---

# Start and goal states
start = [1, 2, 3, 4, 0, 5, 7, 8, 6]
goal = [1, 2, 3, 4, 5, 6, 7, 8, 0]

solution = a_star_8_puzzle(start, goal)

if solution:
    # This is the line that was changed:
    print(f"Solved in {len(solution) - 1} moves! Here are the steps:")
    for i, step in enumerate(solution):
        print(f"\nStep {i}:")
        visualize_puzzle(step) # Use the new function
else:
    print("No solution found.")

Solved in 2 moves! Here are the steps:

Step 0:
-------
| 1 2 3 |
| 4   5 |
| 7 8 6 |
-------

Step 1:
-------
| 1 2 3 |
| 4 5   |
| 7 8 6 |
-------

Step 2:
-------
| 1 2 3 |
| 4 5 6 |
| 7 8   |
-------


In [ ]:
import math

# --- Core Game Functions ---

def print_board(board):
    """Prints the 3x3 board."""
    print()
    for i in range(3):
        print(f" {board[3*i]} | {board[3*i+1]} | {board[3*i+2]} ")
        if i < 2:
            print("---+---+---")
    print()

def check_winner(board):
    """Checks if there is a winner."""
    win_conditions = [
        (0,1,2), (3,4,5), (6,7,8),  # rows
        (0,3,6), (1,4,7), (2,5,8),  # columns
        (0,4,8), (2,4,6)            # diagonals
    ]
    for (x, y, z) in win_conditions:
        if board[x] == board[y] == board[z] and board[x] != ' ':
            return board[x]
    return None

def is_full(board):
    """Checks if the board is full."""
    return ' ' not in board

# --- AI Algorithm (Minimax) ---

def minimax(board, depth, is_maximizing):
    """
    Minimax algorithm to determine the score of a move.
    'O' is the maximizing player (AI, +1)
    'X' is the minimizing player (Human, -1)
    """
    winner = check_winner(board)
    if winner == 'O':
        return 1
    elif winner == 'X':
        return -1
    elif is_full(board):
        return 0

    if is_maximizing:
        best_score = -math.inf
        for i in range(9):
            if board[i] == ' ':
                board[i] = 'O'
                score = minimax(board, depth + 1, False)
                board[i] = ' '
                best_score = max(best_score, score)
        return best_score
    else: # Minimizing
        best_score = math.inf
        for i in range(9):
            if board[i] == ' ':
                board[i] = 'X'
                score = minimax(board, depth + 1, True)
                board[i] = ' '
                best_score = min(best_score, score)
        return best_score

def ai_move(board):
    """
    Finds the best move for the AI ('O') using Minimax.
    """
    best_score = -math.inf
    move = None
    for i in range(9):
        if board[i] == ' ':
            board[i] = 'O'
            score = minimax(board, 0, False) # Find score for minimizing player
            board[i] = ' '
            if score > best_score:
                best_score = score
                move = i
    return move

# --- Human Input Function ---

def get_human_move(board, player):
    """
    Asks a human player for a move and validates it.
    """
    while True:
        try:
            move = input(f"Player '{player}', enter your move (0-8): ")
            move = int(move)
            if 0 <= move <= 8:
                if board[move] == ' ':
                    return move
                else:
                    print("Invalid move. That spot is already taken.")
            else:
                print("Invalid move. Please enter a number between 0 and 8.")
        except ValueError:
            print("Invalid input. Please enter a number.")

# --- Main Game Function (Modified) ---

def play_game():
    board = [' '] * 9
    current_player = 'X' # 'X' always starts
    print("Welcome to Tic-Tac-Toe!")

    # Game mode selection
    mode = ''
    while mode not in ['1', '2']:
        mode = input("Choose a mode:\n 1: Human vs. AI\n 2: Human vs. Human\nEnter 1 or 2: ")
        if mode not in ['1', '2']:
            print("Invalid choice. Please try again.")

    print_board(board)

    # Main game loop
    while True:
        # Get the move
        if current_player == 'X':
            move = get_human_move(board, 'X')
        else: # 'O's turn
            if mode == '1': # AI mode
                print("AI ('O') is thinking...")
                move = ai_move(board)
            else: # Human vs. Human mode
                move = get_human_move(board, 'O')

        # Apply the move
        board[move] = current_player
        print_board(board)

        # Check game status
        winner = check_winner(board)
        if winner:
            print(f"*** Player '{winner}' wins! ***")
            break
        elif is_full(board):
            print("*** It's a draw! ***")
            break

        # Switch player
        current_player = 'O' if current_player == 'X' else 'X'

# --- Run the game ---
play_game()

Welcome to Tic-Tac-Toe!
Choose a mode:
 1: Human vs. AI
 2: Human vs. Human
Enter 1 or 2: 1

   |   |   
---+---+---
   |   |   
---+---+---
   |   |   

Player 'X', enter your move (0-8): 0

 X |   |   
---+---+---
   |   |   
---+---+---
   |   |   

AI ('O') is thinking...

 X |   |   
---+---+---
   | O |   
---+---+---
   |   |   

Player 'X', enter your move (0-8): 8

 X |   |   
---+---+---
   | O |   
---+---+---
   |   | X 

AI ('O') is thinking...

 X | O |   
---+---+---
   | O |   
---+---+---
   |   | X 

Player 'X', enter your move (0-8): 7

 X | O |   
---+---+---
   | O |   
---+---+---
   | X | X 

AI ('O') is thinking...

 X | O |   
---+---+---
   | O |   
---+---+---
 O | X | X 

Player 'X', enter your move (0-8): 2

 X | O | X 
---+---+---
   | O |   
---+---+---
 O | X | X 

AI ('O') is thinking...

 X | O | X 
---+---+---
   | O | O 
---+---+---
 O | X | X 

Player 'X', enter your move (0-8): 1
Invalid move. That spot is already taken.
Player 'X', enter your move

In [ ]:
import sys

def print_pegs(pegs, total_disks):
    """
    Visualizes the current state of the pegs.
    """
    # Determine the maximum width for centering
    max_width = total_disks * 2 - 1

    # Create display-ready lists for each peg
    display_pegs = {}
    for peg_name in ['A', 'B', 'C']:
        display_pegs[peg_name] = []

        # Add empty "air" spots
        num_empty = total_disks - len(pegs[peg_name])
        for _ in range(num_empty):
            display_pegs[peg_name].append( "|" )

        # Add the disks, starting from the top of the stack (end of the list)
        for disk_num in reversed(pegs[peg_name]):
            # Create a string representation of the disk
            disk_str = "#" * (disk_num * 2 - 1)
            display_pegs[peg_name].append(disk_str)

    # Print the pegs level by level, from top to bottom
    print() # Add a blank line for spacing
    for i in range(total_disks):
        line = ""
        for peg_name in ['A', 'B', 'C']:
            # Center the disk/pole string for a neat column
            line += display_pegs[peg_name][i].center(max_width) + "  "
        print(line)

    # Print the base
    base_line = "=" * max_width
    print(f"{base_line}  {base_line}  {base_line}")
    print(f"{'A'.center(max_width)}  {'B'.center(max_width)}  {'C'.center(max_width)}\n")

# Modified Tower of Hanoi function
def tower_of_hanoi_visual(n, source, auxiliary, destination, pegs, total_disks):
    """
    Recursive function to solve and visualize Tower of Hanoi.

    Parameters:
    - n: The current disk to move
    - source: The source peg label ('A', 'B', or 'C')
    - auxiliary: The auxiliary peg label
    - destination: The destination peg label
    - pegs: The dictionary holding the state of the pegs
    - total_disks: The total number of disks (for printing)
    """
    if n == 0:
        # Base case: 0 disks to move, do nothing
        return

    # 1. Move n-1 disks from source to auxiliary
    tower_of_hanoi_visual(n - 1, source, destination, auxiliary, pegs, total_disks)

    # 2. Move the nth disk (the one at the top of the source peg)
    #    from source to destination

    # --- This is the "move" logic ---
    disk = pegs[source].pop()
    pegs[destination].append(disk)
    # --- End of move logic ---

    # Print the action and the resulting state
    print(f"Move disk {n} from {source} -> {destination}")
    print_pegs(pegs, total_disks)

    # 3. Move the n-1 disks from auxiliary to destination
    tower_of_hanoi_visual(n - 1, auxiliary, source, destination, pegs, total_disks)

# --- Main execution ---
n = 3

# Create the initial state of the pegs
# The lists represent the pegs, from bottom to top.
# e.g., [3, 2, 1] means 3 is at the bottom, 1 is at the top.
pegs = {
    'A': list(range(n, 0, -1)), # All disks start on peg A
    'B': [],
    'C': []
}

print(f"Tower of Hanoi solution for {n} disks:\n")

# Print the starting position
print("Initial State:")
print_pegs(pegs, n)

# Run the solution
tower_of_hanoi_visual(n, 'A', 'B', 'C', pegs, n)

Tower of Hanoi solution for 3 disks:

Initial State:

  #      |      |    
 ###     |      |    
#####    |      |    
=====  =====  =====
  A      B      C  

Move disk 1 from A -> C

  |      |      |    
 ###     |      |    
#####    |      #    
=====  =====  =====
  A      B      C  

Move disk 2 from A -> B

  |      |      |    
  |      |      |    
#####   ###     #    
=====  =====  =====
  A      B      C  

Move disk 1 from C -> B

  |      |      |    
  |      #      |    
#####   ###     |    
=====  =====  =====
  A      B      C  

Move disk 3 from A -> C

  |      |      |    
  |      #      |    
  |     ###   #####  
=====  =====  =====
  A      B      C  

Move disk 1 from B -> A

  |      |      |    
  |      |      |    
  #     ###   #####  
=====  =====  =====
  A      B      C  

Move disk 2 from B -> C

  |      |      |    
  |      |     ###   
  #      |    #####  
=====  =====  =====
  A      B      C  

Move disk 1 from A -> C

  |      |      #    


In [ ]:
from collections import deque

def print_jugs(a, b, cap1, cap2):
    """
    Prints an ASCII visualization of the two jugs.
    - a: current amount in jug1
    - b: current amount in jug2
    - cap1: capacity of jug1
    - cap2: capacity of jug2
    """
    print(f"  State: ({a}, {b})")

    # Create lists of strings for each jug's lines
    lines1 = []
    lines2 = []

    # Fill jug 1 lines (from top to bottom)
    for i in range(cap1, 0, -1):
        if i <= a:
            lines1.append("|===|")
        else:
            lines1.append("|   |")

    # Fill jug 2 lines (from top to bottom)
    for i in range(cap2, 0, -1):
        if i <= b:
            lines2.append("|===|")
        else:
            lines2.append("|   |")

    # Normalize heights by padding the shorter jug with empty space
    max_height = max(cap1, cap2)
    while len(lines1) < max_height:
        lines1.insert(0, "     ")

    while len(lines2) < max_height:
        lines2.insert(0, "     ")

    # Print the jugs side-by-side
    for i in range(max_height):
        print(f"  {lines1[i]}   {lines2[i]}")

    # Print the base
    print(f"  +---+   +---+")
    print(f"  Jug 1   Jug 2")
    print(f"  ({cap1}L)    ({cap2}L)")
    print("-" * 20)


def water_jug_problem(jug1, jug2, target):
    """
    Finds the shortest path to the target amount.
    """
    visited = set()

    # The queue will now store: ( (current_a, current_b), path_list )
    queue = deque([ ((0, 0), [(0, 0)]) ])

    while queue:
        # Unpack the current state and the path to get here
        (a, b), path = queue.popleft()

        if (a, b) in visited:
            continue
        visited.add((a, b))

        # --- Goal Check ---
        if a == target or b == target:
            print(f"Target {target} liters reached!")
            return path  # Return the full path

        # --- Possible Operations ---
        next_states = [
            (jug1, b),      # Fill jug1
            (a, jug2),      # Fill jug2
            (0, b),         # Empty jug1
            (a, 0),         # Empty jug2
            (a - min(a, jug2 - b), b + min(a, jug2 - b)),  # Pour jug1 -> jug2
            (a + min(b, jug1 - a), b - min(b, jug1 - a))   # Pour jug2 -> jug1
        ]

        for state in next_states:
            if state not in visited:
                # Add the new state and the path *to* that state
                queue.append((state, path + [state]))

    return None # No solution found

# --- Main Execution ---
jug1_cap = 4
jug2_cap = 3
target_amt = 2

print(f"Solving for J1={jug1_cap}L, J2={jug2_cap}L, Target={target_amt}L\n")
solution_path = water_jug_problem(jug1_cap, jug2_cap, target_amt)

if solution_path:
    print("\n--- Solution Path ---")
    for i, (a, b) in enumerate(solution_path):
        print(f"Step {i}:")
        # Call the visualization function for each step
        print_jugs(a, b, jug1_cap, jug2_cap)
else:
    print("No solution found.")

Solving for J1=4L, J2=3L, Target=2L

Target 2 liters reached!

--- Solution Path ---
Step 0:
  State: (0, 0)
  |   |        
  |   |   |   |
  |   |   |   |
  |   |   |   |
  +---+   +---+
  Jug 1   Jug 2
  (4L)    (3L)
--------------------
Step 1:
  State: (0, 3)
  |   |        
  |   |   |===|
  |   |   |===|
  |   |   |===|
  +---+   +---+
  Jug 1   Jug 2
  (4L)    (3L)
--------------------
Step 2:
  State: (3, 0)
  |   |        
  |===|   |   |
  |===|   |   |
  |===|   |   |
  +---+   +---+
  Jug 1   Jug 2
  (4L)    (3L)
--------------------
Step 3:
  State: (3, 3)
  |   |        
  |===|   |===|
  |===|   |===|
  |===|   |===|
  +---+   +---+
  Jug 1   Jug 2
  (4L)    (3L)
--------------------
Step 4:
  State: (4, 2)
  |===|        
  |===|   |   |
  |===|   |===|
  |===|   |===|
  +---+   +---+
  Jug 1   Jug 2
  (4L)    (3L)
--------------------
